In [10]:
# Import libs
import polars as p
import os

# Declare Global Variable
dir_root  = globals()["_dh"][0]
dir_data_geonames = f"{dir_root}/../data/genere/geonames"
dir_data_result = f"{dir_root}/../data/result/geonames"

file_result_ville_parquet = f"{dir_data_result}/ville_geo.parquet"
file_result_ville_delta = f"{dir_data_result}/ville_geo_delta"

In [11]:
# Create result folder if not exists
if (not(os.path.exists(dir_data_result))):
    os.makedirs(dir_data_result)

In [15]:
# Read data from CSV Files
df_admin2 = p.read_csv(source=f"{dir_data_geonames}/admin2Codes.csv"
                   ,separator=';'
                   ,has_header=True)

df_admin1 = p.read_csv(source=f"{dir_data_geonames}/admin1CodesASCII.csv"
                   ,separator=';'
                   ,has_header=True)

df_FR = p.read_csv(source=f"{dir_data_geonames}/FR.csv"
                   ,separator=';'
                   ,has_header=True
                   ,ignore_errors=False
                   ,schema_overrides={"admin1_code": p.String, "dem" : p.String, "modification_date" : p.Date})



In [16]:
# Get dataframe Schema
print(f"Admin1 : {df_admin1.schema}")
print(f"Admin2 : {df_admin2.schema}")
print(f"FR : {df_FR.schema}")


Admin1 : OrderedDict([('code', String), ('name', String), ('name_ascii', String), ('geonameid', Int64)])
Admin2 : OrderedDict([('codes', String), ('name', String), ('name_ascii', String), ('geonameId', Int64)])
FR : OrderedDict([('geonameid', Int64), ('name', String), ('ascii_name', String), ('alternate_names', String), ('latitude', Float64), ('longitude', Float64), ('feature_class', String), ('feature_code', String), ('country_code', String), ('cc2', String), ('admin1_code', String), ('admin2_code', String), ('admin3_code', String), ('admin4_code', String), ('population', Int64), ('elevation', Int64), ('dem', String), ('timezone', String), ('modification_date', Date)])


In [18]:
# Get the first two rows
df_FR.head(2)


geonameid,name,ascii_name,alternate_names,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
i64,str,str,str,f64,f64,str,str,str,str,str,str,str,str,i64,i64,str,str,date
2659086,"""Col de Recon""","""Col de Recon""","""Rapenaz Col de,Recon Col de""",46.30352,6.82838,"""T""","""PASS""","""FR""","""CH""","""84""","""74""","""744""","""74058""",0,null,"""1733""","""Europe/Paris""",2019-02-15
2659815,"""Lucelle""","""Lucelle""","""La Lucelle Riviere,La Lucelle …",47.43813,7.3428,"""H""","""STM""","""FR""",null,"""00""",null,null,null,0,null,"""533""","""Europe/Zurich""",2022-08-09


In [19]:
# Get summary information
df_FR.describe()

statistic,geonameid,name,ascii_name,alternate_names,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
str,f64,str,str,str,f64,f64,str,str,str,str,str,str,str,str,f64,f64,str,str,str
"""count""",171596.0,"""171596""","""171596""","""109735""",171596.0,171596.0,"""171596""","""171596""","""171596""","""421""","""171386""","""166278""","""158282""","""157895""",171596.0,5457.0,"""171596""","""171504""","""171596"""
"""null_count""",0.0,"""0""","""0""","""61861""",0.0,0.0,"""0""","""0""","""0""","""171175""","""210""","""5318""","""13314""","""13701""",0.0,166139.0,"""0""","""92""","""0"""
"""mean""",6.3091e6,null,null,null,46.710248,2.633993,null,null,null,null,null,null,null,null,2654.276388,1305.433755,null,null,"""2018-05-28"""
"""std""",3.4308e6,null,null,null,2.19703,2.997941,null,null,null,null,null,null,null,null,174786.100996,939.095979,null,null,null
"""min""",2.659086e6,"""(Former) Roman Catholic Dioces…","""(Former) Roman Catholic Dioces…","""(mthf alfn almʿasr (qlʿt mwnsw…",4.5422,-109.23333,"""A""","""ADM1""","""FR""","""AD""","""0""","""1""","""11""","""1001""",0.0,0.0,"""-1""","""Europe/Amsterdam""","""1994-01-06"""
"""25%""",3.010133e6,null,null,null,44.97226,0.62247,null,null,null,null,null,null,null,null,0.0,544.0,null,null,"""2016-02-18"""
"""50%""",6.436627e6,null,null,null,46.99443,2.48527,null,null,null,null,null,null,null,null,0.0,1155.0,null,null,"""2019-03-23"""
"""75%""",8.294478e6,null,null,null,48.61667,4.86755,null,null,null,null,null,null,null,null,313.0,1958.0,null,null,"""2019-06-08"""
"""max""",1.2908801e7,"""Œutrange""","""water tower Moret""","""¨Punta de Lliterola""",51.23104,9.75,"""V""","""ZOO""","""FR""","""LU,FR""","""C1""","""95""","""953""","""95690""",6.6987244e7,4810.0,"""999""","""Pacific/Marquesas""","""2024-06-17"""


In [20]:
# Get the 5 first rows from admin1
df_admin1.head(n=5)

code,name,name_ascii,geonameid
str,str,str,i64
"""AD.06""","""Sant Julià de Loria""","""Sant Julia de Loria""",3039162
"""AD.05""","""Ordino""","""Ordino""",3039676
"""AD.04""","""La Massana""","""La Massana""",3040131
"""AD.03""","""Encamp""","""Encamp""",3040684
"""AD.02""","""Canillo""","""Canillo""",3041203


In [21]:
# Get the 5 first rows from admin2
df_admin2.head(n=5)

codes,name,name_ascii,geonameId
str,str,str,i64
"""AE.01.101""","""Abu Dhabi Municipality""","""Abu Dhabi Municipality""",12047239
"""AE.01.102""","""Al Ain Municipality""","""Al Ain Municipality""",12047240
"""AE.01.103""","""Al Dhafra""","""Al Dhafra""",12047241
"""AE.04.701""","""Al Fujairah Municipality""","""Al Fujairah Municipality""",12047242
"""AE.04.702""","""Dibba Al Fujairah Municipality""","""Dibba Al Fujairah Municipality""",12047243


In [22]:
# Retrieve information from FR data (with filter, select, and new column)
df_FR.filter((p.col("population") > 20000) & (~ p.col("admin2_code").is_in(["2B","2A"]))) \
    .select(p.col("geonameid","name","country_code","admin2_code","population","modification_date")) \
    .with_columns(p.col("admin2_code").cast(p.Int64).alias("departement_int"))

geonameid,name,country_code,admin2_code,population,modification_date,departement_int
i64,str,str,str,i64,date,i64
2967196,"""Yvelines""","""FR""","""78""",1433447,2018-02-06,78
2967202,"""Arrondissement d'Yssingeaux""","""FR""","""43""",85073,2023-03-14,43
2967222,"""Yonne""","""FR""","""89""",337504,2022-03-10,89
2967245,"""Yerres""","""FR""","""91""",28897,2016-02-18,91
2967324,"""Arrondissement de Wissembourg""","""FR""","""67""",70020,2019-03-27,67
…,…,…,…,…,…,…
12808664,"""Bel-Air""","""FR""","""75""",36279,2024-04-25,75
12808673,"""Paris 13e Arrondissement""","""FR""","""75""",177833,2024-04-26,75
12808674,"""Croulebarbe""","""FR""","""94""",20062,2024-04-26,94


In [23]:
# Retrieve information from FR data with a Over and Sort
df_FR.filter(p.col("admin4_code").is_not_null()) \
    .with_columns(p.col("population").max().over("admin2_code").alias("testing")) \
    .sort(by=[p.col("admin2_code"),p.col("population")],descending=[True,False]).head(10)

geonameid,name,ascii_name,alternate_names,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date,testing
i64,str,str,str,f64,f64,str,str,str,str,str,str,str,str,i64,i64,str,str,date,i64
12495506,"""Rue Pierre Butin""","""Rue Pierre Butin""",null,49.04945,2.09556,"""R""","""ST""","""FR""",null,"""11""","""95""","""953""","""95500""",0,null,"""25""","""Europe/Paris""",2022-10-28,183430
9177883,"""Marriott Paris Cdg Airport Hot""","""Marriott Paris Cdg Airport Hot""",null,49.00498,2.52092,"""S""","""HTL""","""FR""",null,"""11""","""95""","""952""","""95527""",0,null,"""107""","""Europe/Paris""",2016-02-18,183430
8286140,"""Pierrelaye Railway Station""","""Pierrelaye Railway Station""","""87276097""",49.01928,2.15403,"""S""","""RSTN""","""FR""",null,"""11""","""95""","""951""","""95488""",0,null,"""50""","""Europe/Paris""",2019-03-26,183430
8286139,"""Montigny-Beauchamp Railway Sta…","""Montigny-Beauchamp Railway Sta…","""87276089""",49.00778,2.19682,"""S""","""RSTN""","""FR""",null,"""11""","""95""","""951""","""95488""",0,null,"""62""","""Europe/Paris""",2019-03-28,183430
8286138,"""Cernay (Val-d'Oise) Railway St…","""Cernay (Val-d'Oise) Railway St…","""87276063""",48.98528,2.25702,"""S""","""RSTN""","""FR""",null,"""11""","""95""","""951""","""95219""",0,null,"""50""","""Europe/Paris""",2019-03-26,183430
8286137,"""Ermont-Eaubonne Railway Statio…","""Ermont-Eaubonne Railway Statio…","""87276055""",48.98054,2.27276,"""S""","""RSTN""","""FR""",null,"""11""","""95""","""951""","""95203""",0,null,"""48""","""Europe/Paris""",2019-03-26,183430
8286136,"""Champ de Courses d'Enghien Rai…","""Champ de Courses d'Enghien Rai…","""87276030""",48.97974,2.29175,"""S""","""RSTN""","""FR""",null,"""11""","""95""","""952""","""95598""",0,null,"""42""","""Europe/Paris""",2016-02-18,183430
8286135,"""Enghien les Bains Railway Stat…","""Enghien les Bains Railway Stat…","""87276022""",48.97353,2.30614,"""S""","""RSTN""","""FR""",null,"""11""","""95""","""952""","""95210""",0,null,"""46""","""Europe/Paris""",2016-02-18,183430
8286126,"""Seugy Railway Station""","""Seugy Railway Station""","""87272039""",49.11918,2.39905,"""S""","""RSTN""","""FR""",null,"""11""","""95""","""952""","""95594""",0,null,"""82""","""Europe/Paris""",2016-02-18,183430


In [24]:
# Retrieve information from FR data with complexe query
df_FR.filter(p.col("admin4_code").is_not_null()) \
    .with_columns(rownumber = (p.col("population").rank(method="ordinal",descending=True).over(expr=["admin2_code"]))) \
    .filter(p.col("rownumber") == 1) \
    .sort(by=["admin2_code"],descending=[True])

geonameid,name,ascii_name,alternate_names,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date,rownumber
i64,str,str,str,f64,f64,str,str,str,str,str,str,str,str,i64,i64,str,str,date,u32
8555643,"""Cergy-Pontoise""","""Cergy-Pontoise""","""Cergy-Pontoise""",49.03894,2.07805,"""P""","""PPLL""","""FR""",null,"""11""","""95""","""953""","""95127""",183430,null,"""54""","""Europe/Paris""",2016-02-18,1
6452043,"""Vitry-sur-Seine""","""Vitry-sur-Seine""","""94081,Vitry-sur-Seine""",48.7875,2.39278,"""A""","""ADM4""","""FR""",null,"""11""","""94""","""943""","""94081""",92755,null,"""42""","""Europe/Paris""",2019-03-29,1
6455341,"""Saint-Denis""","""Saint-Denis""","""93066,Kamuna Sen-Dehni,Sain-Nt…",48.93556,2.35389,"""A""","""ADM4""","""FR""",null,"""11""","""93""","""933""","""93066""",111354,null,"""34""","""Europe/Paris""",2019-03-29,1
6451962,"""Boulogne-Billancourt""","""Boulogne-Billancourt""","""92012""",48.83528,2.24139,"""A""","""ADM4""","""FR""",null,"""11""","""92""","""923""","""92012""",119645,null,"""38""","""Europe/Paris""",2019-03-23,1
6454878,"""Évry-Courcouronnes""","""Evry-Courcouronnes""","""91228,Aebura,Eburiacum,Ehvri,E…",48.63389,2.44417,"""A""","""ADM4""","""FR""",null,"""11""","""91""","""912""","""91228""",68090,null,"""74""","""Europe/Paris""",2019-03-29,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3016702,"""Gap""","""Gap""","""GAT,Gap,Vapincum,gap,gapeu,ghb…",44.55858,6.07868,"""P""","""PPLA2""","""FR""",null,"""93""","""05""","""052""","""05061""",42715,null,"""741""","""Europe/Paris""",2019-08-07,1
6425528,"""Manosque""","""Manosque""","""04112,Manosque""",43.83333,5.78333,"""A""","""ADM4""","""FR""",null,"""93""","""04""","""044""","""04112""",21868,null,"""390""","""Europe/Paris""",2019-03-29,1
2992292,"""Montluçon""","""Montlucon""","""MCU,Monlison,Monljuson,Mons Lu…",46.34015,2.60254,"""P""","""PPLA3""","""FR""",null,"""84""","""03""","""031""","""03185""",44960,null,"""208""","""Europe/Paris""",2019-04-10,1


In [25]:
# Retrieve information if the world "palaiseau" exist in the name
df_FR.filter(p.col("name").str.to_lowercase().str.contains_any(["palaiseau"]))

geonameid,name,ascii_name,alternate_names,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
i64,str,str,str,f64,f64,str,str,str,str,str,str,str,str,i64,i64,str,str,date
2988758,"""Palaiseau""","""Palaiseau""","""Palaiseau,Palezo,Paliseau,baly…",48.71828,2.2498,"""P""","""PPLA3""","""FR""",null,"""11""","""91""","""913""","""91477""",31987,null,"""92""","""Europe/Paris""",2019-04-10
6446184,"""Palaiseau""","""Palaiseau""","""91477,Palaiseau""",48.71833,2.24972,"""A""","""ADM4""","""FR""",null,"""11""","""91""","""913""","""91477""",34120,null,"""92""","""Europe/Paris""",2019-03-29
6457367,"""Arrondissement de Palaiseau""","""Arrondissement de Palaiseau""","""Palaiseau""",48.7145,2.2484,"""A""","""ADM3""","""FR""",null,"""11""","""91""","""913""",null,620608,null,"""72""","""Europe/Paris""",2019-03-29
6508490,"""Novotel Massy Palaiseau""","""Novotel Massy Palaiseau""",null,48.7192,2.2637,"""S""","""HTL""","""FR""",null,"""11""","""91""","""913""","""91377""",0,null,"""96""","""Europe/Paris""",2019-03-28
8286672,"""Massy-Palaiseau-Grande-Ceintur…","""Massy-Palaiseau-Grande-Ceintur…","""87393579""",48.72412,2.25594,"""S""","""RSTN""","""FR""",null,"""11""","""91""","""913""","""91377""",0,null,"""86""","""Europe/Paris""",2016-02-18
8297859,"""Bureau de Poste de Palaiseau""","""Bureau de Poste de Palaiseau""",null,48.71247,2.24586,"""S""","""PO""","""FR""",null,"""11""","""91""","""913""","""91477""",0,null,"""73""","""Europe/Paris""",2016-02-18
8297904,"""Bureau de Poste de Palaiseau L…","""Bureau de Poste de Palaiseau L…",null,48.7055,2.21047,"""S""","""PO""","""FR""",null,"""11""","""91""","""913""","""91477""",0,null,"""82""","""Europe/Paris""",2016-02-18
9178654,"""Relais De Palaiseau""","""Relais De Palaiseau""",null,48.70943,2.25456,"""S""","""HTL""","""FR""",null,"""11""","""91""","""913""","""91477""",0,null,"""55""","""Europe/Paris""",2016-02-18


In [26]:
# Extract all the cities from FR Data
df_villes = df_FR.filter(p.col("feature_code")=="ADM4") \
                    .with_columns(p.concat_list([p.col("country_code"),p.col("admin1_code"),p.col("admin2_code")]).list.join(".").alias("codes_ADM2")) \
                    .with_columns(p.concat_list([p.col("country_code"),p.col("admin1_code")]).list.join(".").alias("codes_ADM1"))

In [27]:
# Extract the French information from admin1 and admin2
df_admin2_FR = df_admin2.filter(p.col("codes").str.starts_with("FR"))
df_admin1_FR = df_admin1.filter(p.col("code").str.starts_with("FR"))

In [28]:
# Get the 5 first row from the cities data
df_villes.head(5)


geonameid,name,ascii_name,alternate_names,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date,codes_ADM2,codes_ADM1
i64,str,str,str,f64,f64,str,str,str,str,str,str,str,str,i64,i64,str,str,date,str,str
2968577,"""Villeneuve-d'Ascq""","""Villeneuve-d'Ascq""","""59009,Lungsod ng Villeneuve-d'…",50.62278,3.14417,"""A""","""ADM4""","""FR""",null,"""32""","""59""","""595""","""59009""",62358,null,"""31""","""Europe/Paris""",2019-03-29,"""FR.32.59""","""FR.32"""
2973014,"""Bitschwiller-lès-Thann""","""Bitschwiller-les-Thann""","""68040,Bitschweiler,Bitshviler …",47.83028,7.08028,"""A""","""ADM4""","""FR""",null,"""44""","""68""","""686""","""68040""",1973,null,"""358""","""Europe/Paris""",2019-03-23,"""FR.44.68""","""FR.44"""
2986293,"""Pontault-Combault""","""Pontault-Combault""","""77373""",48.798,2.6063,"""A""","""ADM4""","""FR""",null,"""11""","""77""","""775""","""77373""",38326,null,"""107""","""Europe/Paris""",2019-03-23,"""FR.11.77""","""FR.11"""
3008501,"""L'Albère""","""L'Albere""","""66001,L'Albere,L'Albère,l'Albe…",42.48278,2.89444,"""A""","""ADM4""","""FR""",null,"""76""","""66""","""661""","""66001""",83,null,"""540""","""Europe/Paris""",2019-03-29,"""FR.76.66""","""FR.76"""
6324452,"""Maureillas-las-Illas""","""Maureillas-las-Illas""","""66106,Maureillas-las-Illas,Mor…",42.49028,2.81028,"""A""","""ADM4""","""FR""",null,"""76""","""66""","""661""","""66106""",2559,null,"""131""","""Europe/Paris""",2019-03-29,"""FR.76.66""","""FR.76"""


In [29]:
# Join admin2 and admin1 with cities data to build a new dataset with cities, department, region and population
df_villes_details = df_villes.join(other=df_admin2_FR.rename(mapping={"name":"name_ADM2"}).select(["codes","name_ADM2"]), left_on="codes_ADM2", right_on="codes", how="left", coalesce=False ) \
                             .join(other=df_admin1_FR.rename(mapping={"name":"name_ADM1"}).select(["code","name_ADM1"]), left_on="codes_ADM1", right_on="code", how="left", coalesce=False)

In [30]:
# Retrieve only the needed information from the result dataframe
df_villes_result = \
    df_villes_details.select(city_name="name"
                         ,department_name="name_ADM2"
                         ,department_code="admin2_code"
                         ,region_name="name_ADM1"
                         ,region_code="admin1_code"
                         ,population="population"
                         ,last_modification="modification_date") \
                .sort(by=["department_code","population"],descending=[False,True])

In [31]:
# Get the schema with pyarrow format
df_villes_result.to_arrow().schema

city_name: large_string
department_name: large_string
department_code: large_string
region_name: large_string
region_code: large_string
population: int64
last_modification: date32[day]

In [32]:
# Write the result with parquet format
df_villes_result.write_parquet(file=file_result_ville_parquet)

In [33]:
# Write the result with delta format (overwrite method)
df_villes_result.write_delta(file_result_ville_delta,mode="overwrite",
    delta_write_options={"schema_mode": "overwrite"},)

In [34]:
# Retrieve the written information from delta format
p.read_delta(file_result_ville_delta)

city_name,department_name,department_code,region_name,region_code,population,last_modification
str,str,str,str,str,i64,date
"""Bourg-en-Bresse""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",41365,2019-03-29
"""Oyonnax""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",22559,2019-03-29
"""Valserhône""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",16302,2019-03-29
"""Ambérieu-en-Bugey""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",14081,2019-03-29
"""Gex""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",12652,2019-03-29
…,…,…,…,…,…,…
"""Châtenay-en-France""","""Val d'Oise""","""95""","""Île-de-France""","""11""",72,2019-03-29
"""Épinay-Champlâtreux""","""Val d'Oise""","""95""","""Île-de-France""","""11""",65,2019-03-29
"""Menouville""","""Val d'Oise""","""95""","""Île-de-France""","""11""",62,2019-03-23


In [40]:
# Retrieve information from parquet format
p.read_parquet(file_result_ville_parquet)

city_name,department_name,department_code,region_name,region_code,population,last_modification
str,str,str,str,str,i64,date
"""Bourg-en-Bresse""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",41365,2019-03-29
"""Oyonnax""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",22559,2019-03-29
"""Valserhône""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",16302,2019-03-29
"""Ambérieu-en-Bugey""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",14081,2019-03-29
"""Gex""","""Ain""","""01""","""Auvergne-Rhône-Alpes""","""84""",12652,2019-03-29
…,…,…,…,…,…,…
"""Châtenay-en-France""","""Val d'Oise""","""95""","""Île-de-France""","""11""",72,2019-03-29
"""Épinay-Champlâtreux""","""Val d'Oise""","""95""","""Île-de-France""","""11""",65,2019-03-29
"""Menouville""","""Val d'Oise""","""95""","""Île-de-France""","""11""",62,2019-03-23
